<table style="width:100%">
  <tr style="background-color:#FFFFFF">
    <td style="width:10%">
        <img src="https://play-lh.googleusercontent.com/K9FC23HoIYy_idt3LL26zrzMchOGORVzLNmi3L5u-MaghdjpdgHAWv7wQlscNmo9wYjC=s180-rw" alt="DiDi" width=120></td>
    <td style="width:10%">
        <img src="https://logodownload.org/wp-content/uploads/2019/05/99-app-logo.png" alt="99" width=120></td>
      <td>
          <h1 style="text-align:left; text-indent :2em">DATA TRANSFORMATION • PART 3</h1>
          <h2 style="text-align:left; text-indent :2.4em;">ISIS SANTOS COSTA • 2020</h2>
      </td>
  </tr>
</table>

## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Resuming: retrieving the DFs

In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
with open('df_orders_t3.csv') as f: cols = f.readline().split(',')
del cols[0]
cols[-1] = cols[-1].replace('\n','')
df_orders = pd.read_csv('df_orders_t3.csv', usecols=cols) # reading previously transformed data
df_orders.head(2)

,order_id,passenger_id,pickup_datetime,pickup_longitude,pickup_latitude,payment_type,pickup_date,pickup_dayofweek,pickup_week,pickup_isweekend,pickup_hour,pickup_isrush,pickup_month,pickup_weekinmonth
0,6433697,5234567816269547,2014-04-11 18:06:45,-74.015684,40.714476,CRD,2014-04-11,Friday,15,0,18,1,4,2
1,6433698,5234567812367422,2014-04-12 00:20:28,-73.958403,40.713247,CRD,2014-04-12,Saturday,15,1,0,0,4,2


In [4]:
with open('df_trips_t3.csv') as f: cols = f.readline().split(',')
del cols[0]
cols[-1] = cols[-1].replace('\n','')
df_trips = pd.read_csv('df_trips_t3.csv', usecols=cols) # reading previously transformed data
df_trips.head(2)

,order_id,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,rate_type,dropoff_longitude,dropoff_latitude,payment_type,...,pickup_dayofweek,pickup_week,pickup_isweekend,pickup_hour,pickup_isrush,duration,duration_minutes,duration_type,pickup_month,pickup_weekinmonth
0,718300,2014-05-24 09:08:43,2014-05-24 09:19:06,2.1,-73.967172,40.769851,Standard,-73.995022,40.765441,CRD,...,Saturday,21,1,9,1,0 days 00:10:23,10.383333,10-20min,5,4
1,14054197,2014-04-10 15:14:12,2014-04-10 15:49:05,12.4,-73.991044,40.749704,Standard,-73.909173,40.823500,CSH,...,Thursday,15,0,15,1,0 days 00:34:53,34.883333,30-40min,4,2


## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Combining the tables: outer join 🔗

⏳ <i> Considering the time to load/save the DFs, and that DF trips has same information as DF orders for the converted trips, let's join them into one

### Listing information added when orders get converted: trips_features

In [5]:
df_orders.columns

Index(['order_id', 'passenger_id', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'payment_type', 'pickup_date', 'pickup_dayofweek',
       'pickup_week', 'pickup_isweekend', 'pickup_hour', 'pickup_isrush',
       'pickup_month', 'pickup_weekinmonth'],
      dtype='object')

In [6]:
df_trips.columns

Index(['order_id', 'pickup_datetime', 'dropoff_datetime', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'rate_type', 'dropoff_longitude',
       'dropoff_latitude', 'payment_type', 'fare_amount', 'tip_amount',
       'total_amount', 'passenger_id', 'pickup_date', 'pickup_dayofweek',
       'pickup_week', 'pickup_isweekend', 'pickup_hour', 'pickup_isrush',
       'duration', 'duration_minutes', 'duration_type', 'pickup_month',
       'pickup_weekinmonth'],
      dtype='object')

In [7]:
# columns to add for orders that became trips
trips_features = set(df_trips.columns) - set(df_orders.columns)
trips_features

{'dropoff_datetime',
 'dropoff_latitude',
 'dropoff_longitude',
 'duration',
 'duration_minutes',
 'duration_type',
 'fare_amount',
 'rate_type',
 'tip_amount',
 'total_amount',
 'trip_distance'}

### Adding trips_features to the table of all orders

In [8]:
trips_features.add('order_id')

In [9]:
df_orders_tF = pd.merge(df_orders, df_trips[trips_features], on='order_id', how='outer')

In [10]:
df_orders_tF.head(3)

,order_id,passenger_id,pickup_datetime,pickup_longitude,pickup_latitude,payment_type,pickup_date,pickup_dayofweek,pickup_week,pickup_isweekend,...,fare_amount,trip_distance,dropoff_longitude,duration,dropoff_datetime,duration_minutes,dropoff_latitude,total_amount,duration_type,tip_amount
0,6433697,5234567816269547,2014-04-11 18:06:45,-74.015684,40.714476,CRD,2014-04-11,Friday,15,0,...,9.5,1.6,-74.000814,0 days 00:11:34,2014-04-11 18:18:19,11.566667,40.718173,13.2,10-20min,2.2
1,6433698,5234567812367422,2014-04-12 00:20:28,-73.958403,40.713247,CRD,2014-04-12,Saturday,15,1,...,22.0,5.5,-73.991395,0 days 00:26:51,2014-04-12 00:47:19,26.850000,40.758526,27.0,20-30min,4.0
2,6433699,5234567814730471,2014-04-11 23:51:28,-73.987125,40.747428,CRD,2014-04-11,Friday,15,0,...,8.0,1.6,-74.005073,0 days 00:08:13,2014-04-11 23:59:41,8.216667,40.740963,10.8,5-10min,1.8


🔎 <i> Checking if trips features are the same as in df_trips, for same order_id:

In [11]:
df_trips.loc[df_trips['order_id']==6433697]

,order_id,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,rate_type,dropoff_longitude,dropoff_latitude,payment_type,...,pickup_dayofweek,pickup_week,pickup_isweekend,pickup_hour,pickup_isrush,duration,duration_minutes,duration_type,pickup_month,pickup_weekinmonth
4218327,6433697,2014-04-11 18:06:45,2014-04-11 18:18:19,1.6,-74.015684,40.714476,Standard,-74.000814,40.718173,CRD,...,Friday,15,0,18,1,0 days 00:11:34,11.566667,10-20min,4,2


<i> Nice, looks good! ✔️

### 🗃️ Reordering the columns of the new DF df_orders_tF (<i>t</i>ransformed, <i>F</i>inal)

👀 <i> Let's organize the columns, allocating related features adjacent to each other, making it easier to compare them. </i> 👀

In [12]:
cols = list(df_orders_tF.columns.values)
cols

['order_id',
 'passenger_id',
 'pickup_datetime',
 'pickup_longitude',
 'pickup_latitude',
 'payment_type',
 'pickup_date',
 'pickup_dayofweek',
 'pickup_week',
 'pickup_isweekend',
 'pickup_hour',
 'pickup_isrush',
 'pickup_month',
 'pickup_weekinmonth',
 'rate_type',
 'fare_amount',
 'trip_distance',
 'dropoff_longitude',
 'duration',
 'dropoff_datetime',
 'duration_minutes',
 'dropoff_latitude',
 'total_amount',
 'duration_type',
 'tip_amount']

In [13]:
cols = ['order_id',
 'passenger_id',
 'pickup_datetime',
 'pickup_month',
 'pickup_week',
 'pickup_weekinmonth',
 'pickup_date',
 'pickup_dayofweek',
 'pickup_isweekend',
 'pickup_hour',
 'pickup_isrush',
 'pickup_latitude',
 'pickup_longitude',
 'dropoff_latitude',
 'dropoff_longitude',
 'dropoff_datetime',
 'duration',
 'duration_minutes',
 'duration_type',
 'trip_distance',
 'total_amount',
 'fare_amount',
 'tip_amount',
 'rate_type',
 'payment_type']
cols

['order_id',
 'passenger_id',
 'pickup_datetime',
 'pickup_month',
 'pickup_week',
 'pickup_weekinmonth',
 'pickup_date',
 'pickup_dayofweek',
 'pickup_isweekend',
 'pickup_hour',
 'pickup_isrush',
 'pickup_latitude',
 'pickup_longitude',
 'dropoff_latitude',
 'dropoff_longitude',
 'dropoff_datetime',
 'duration',
 'duration_minutes',
 'duration_type',
 'trip_distance',
 'total_amount',
 'fare_amount',
 'tip_amount',
 'rate_type',
 'payment_type']

In [14]:
df_orders_tF = df_orders_tF[cols]
df_orders_tF.head()

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,dropoff_datetime,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,2014-04-11 18:18:19,0 days 00:11:34,11.566667,10-20min,1.6,13.20,9.5,2.20,Standard,CRD
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,2014-04-12 00:47:19,0 days 00:26:51,26.850000,20-30min,5.5,27.00,22.0,4.00,Standard,CRD
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,...,2014-04-11 23:59:41,0 days 00:08:13,8.216667,5-10min,1.6,10.80,8.0,1.80,Standard,CRD
3,6433700,5234567812353703,2014-04-12 09:14:58,4,15,2,2014-04-12,Saturday,1,9,...,2014-04-12 09:24:44,0 days 00:09:46,9.766667,5-10min,1.3,10.20,8.0,1.70,Standard,CRD
4,6433701,5234567822243871,2014-04-12 18:59:24,4,15,2,2014-04-12,Saturday,1,18,...,2014-04-12 19:21:31,0 days 00:22:07,22.116667,20-30min,3.7,21.25,16.5,4.25,Standard,CRD


<b>(</b>  Θ_Θ<b>)</b> <i>  &nbsp;&nbsp;&nbsp; Nice </i>&nbsp;&nbsp;&nbsp;<b>(</b>  Θ_Θ<b>)</b>

## Tʀᴀɴsғᴏʀᴍᴀᴛɪᴏɴ  •  Feature engineering: converting 📍coordinates into 🗺️neighborhoods

### Preparing for heavy processing: exporting the dataframe as transformed so far 📤

<i> In case the Kernel... leaves us </i>  &nbsp;&nbsp; • ‖ •  &nbsp;&nbsp; [ Kernel ] 🎒😎➡️👋

In [23]:
df_orders_tF.to_csv('df_orders_tF.csv')

time: 12min 39s


### Preparing for heavy processing: autotime + tqdm (progress bar)

In [1]:
#!pip install ipython-autotime
%load_ext autotime

time: 0 ns


In [2]:
import tqdm
from tqdm.notebook import tqdm_notebook

time: 3.99 s


### Importing: Geopandas • Geopy (Nominatim+RateLimiter) • PyPlot • Plotly_express

In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

time: 51 s


In [4]:
#!pip install plotly_express==0.4.0
import plotly_express as px

time: 40.5 s


In [5]:
#!pip install geopy
import geopy
from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter

time: 2.11 s


In [6]:
#!pip install wheel
#!pip install pipwin

#!pipwin install numpy
#!pipwin install pandas
#!pipwin install shapely
#!pipwin install gdal
#!pipwin install fiona
#!pipwin install pyproj
#!pipwin install six
#!pipwin install rtree
#!pipwin install geopandas

time: 0 ns


In [7]:
#!pip install geopandas # <<== This approach did't work, had to run the cells above
#!conda install -c conda-forge geopandas # <<== This approach did't work, had to run the cells above
import geopandas as gpd

time: 21.8 s


### Constructing Geocoder

In [8]:
locator = Nominatim (user_agent='myGeocoder', timeout=24000) # extended timeout

time: 0 ns


### Reverse geocoding • Pilot test ✔️

📝 <i> It took some testing to figure out the administrative organization of the city and to find out how to return results in spite of missing data.
    <p> <b>Regarding missing data</b> </i>🕳️<i>: not all the addresses in the reference presented information for all of the queried fields, and when a missing value was reached, the whole fetching was being interrupted. The solution was to fetch the entire address - a dictionary - and then <u>filter</u> the needed data from 'address' in the DF, applying the method <u>setdefault</u> to the dictionaries.</p>
    <p> <b>Regarding the division of the city</b> </i>🗺️<i>: the initial intent to dismember the city into neighborhoods led to choosing <u>'neighbourhood'</u> from the geocoder address. It happened that on one side it presented a lot of missing data and, on the other side, there seemed to be too many unique values: dividing into too many districts wouldn't ease the analysis any much. Its bigger container - <u>'suburb'</u> - has been thus added to the study.

In [28]:
df_orders_geo = df_orders_tF[['order_id', 'pickup_latitude', 'pickup_longitude']][0:12]

df_orders_geo['address'] = df_orders_geo.apply(lambda row: locator.reverse((row['pickup_latitude'], row['pickup_longitude'])).raw['address'], axis=1)

df_orders_geo['neighborhood'] = df_orders_geo['address']
df_orders_geo['suburb'] = df_orders_geo['address']

for i in range(len(df_orders_geo)):
    df_orders_geo['address'][i].setdefault('neighbourhood')
    df_orders_geo['neighborhood'][i] = df_orders_geo['address'][i]['neighbourhood']
    
    df_orders_geo['address'][i].setdefault('suburb')
    df_orders_geo['suburb'][i] = df_orders_geo['address'][i]['suburb']
    
df_orders_geo.head(3)

,order_id,pickup_latitude,pickup_longitude,address,neighborhood,suburb
0,6433697,40.714476,-74.015684,"{'house_number': '4', 'road': 'Vesey Street', ...",Battery Park City,Manhattan
1,6433698,40.713247,-73.958403,"{'house_number': '175', 'road': 'Roebling Stre...",Williamsburg,Brooklyn
2,6433699,40.747428,-73.987125,"{'house_number': '25', 'road': 'West 31st Stre...",Koreatown,Manhattan


time: 8.61 s


In [29]:
100 * df_orders_geo['neighborhood'].value_counts() / len(df_orders_geo)

Midtown East          16.666667
Battery Park City      8.333333
Carnegie Hill          8.333333
Financial District     8.333333
Williamsburg           8.333333
Koreatown              8.333333
Lenox Hill             8.333333
Greenwich Village      8.333333
Yorkville              8.333333
Midtown South          8.333333
Name: neighborhood, dtype: float64

time: 0 ns


In [30]:
100 * df_orders_geo['suburb'].value_counts() / len(df_orders_geo)

Manhattan    83.333333
Brooklyn     16.666667
Name: suburb, dtype: float64

time: 1.44 s


In [31]:
100 * df_orders_geo.loc[df_orders_geo['suburb']=='Manhattan', 'neighborhood'].value_counts() / len(df_orders_geo)

Midtown East          16.666667
Battery Park City      8.333333
Carnegie Hill          8.333333
Financial District     8.333333
Koreatown              8.333333
Lenox Hill             8.333333
Greenwich Village      8.333333
Yorkville              8.333333
Midtown South          8.333333
Name: neighborhood, dtype: float64

time: 438 ms


### Reverse geocoding • Full scale 📈: 1<sup>st</sup> trial with Nominatim, by list ❌

<i> The following code has been tried earlier, but kept getting interrupted before completion: </i>

In [ ]:
# rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.0002)

In [ ]:
# tqdm_notebook.pandas()

# df_orders['neighborhood'] = df_orders['pickup_coordinates'].progress_apply(rgeocode)
# df_orders.head(2)'''

### Reverse geocoding • Full scale 📈: n<sup>th</sup> trial with Nominatim, by list ❌

<i> After many trials (and some Kernels lost down the road), decided the following would be the last. <br>
    On hitting a missing value, the whole processing for the other rows has been lost (msg in the collapsed cells). </i>

In [32]:
df_orders_tF['pickup_address'] = df_orders_tF.apply(lambda row: locator.reverse((row['pickup_latitude'], row['pickup_longitude'])).raw['address'], axis=1)

TypeError: `address` must not be None

time: 4h 9min 3s


In [33]:
df_orders_tF['pickup_address'].head()

KeyError: 'pickup_address'

time: 2.31 s


<i> First reaction: 'Yaaay! Just four hours \o/' </i>😃<i> <br> 
    After checking with '.head()' </i>😒
    <p> <i> Hm... So, fetching was interrupted when a missing value was reached. </i>

### ( Resuming after Kernel shut down • Retrieving data: nice it'd been saved! 😅 )

In [9]:
import pandas as pd

time: 0 ns


In [10]:
import warnings
warnings.filterwarnings('ignore')

time: 0 ns


In [11]:
df_orders_tF = pd.read_csv('df_orders_tF.csv', index_col=0) # reading previously transformed data
df_orders_tF.head(2)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,dropoff_datetime,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,2014-04-11 18:18:19,0 days 00:11:34,11.566667,10-20min,1.6,13.2,9.5,2.2,Standard,CRD
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,2014-04-12 00:47:19,0 days 00:26:51,26.850000,20-30min,5.5,27.0,22.0,4.0,Standard,CRD


time: 8min 41s


### Reverse geocoding • Full scale 📈 [ try / except ] ❌

<i> The following approach adds an 'antidote' for interrupting the query when is to define a 'ignore Exception', the beautiful pair try / except:

In [12]:
try:
    df_orders_tF['pickup_address'] = df_orders_tF.apply(lambda row: locator.reverse((row['pickup_latitude'], row['pickup_longitude'])).raw['address'], axis=1)
except Exception:
    pass

time: 3h 55min 10s


<i> Again, in case the Kernel is gone:

In [13]:
df_orders_tF.to_csv('df_orders_tF.csv')

time: 43min 5s


In [14]:
df_orders_tF.head(3)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,dropoff_datetime,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,2014-04-11 18:18:19,0 days 00:11:34,11.566667,10-20min,1.6,13.2,9.5,2.2,Standard,CRD
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,2014-04-12 00:47:19,0 days 00:26:51,26.850000,20-30min,5.5,27.0,22.0,4.0,Standard,CRD
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,...,2014-04-11 23:59:41,0 days 00:08:13,8.216667,5-10min,1.6,10.8,8.0,1.8,Standard,CRD


time: 18.7 s


In [15]:
df_orders_tF['pickup_address'].head(3)

KeyError: 'pickup_address'

time: 1min 32s


In [16]:
df_orders_tF.shape

(15000000, 25)

time: 203 ms


<i> Learning how the combo [ try / expect + apply ] operate, the painful way... Ok, this leads us to the next trial: item by item. </i>

### Reverse geocoding • Nominatim [ try / except ] item by item ✔️ (kinda... 90 days!)

In [48]:
df_orders_tF['address'] = [''] * len(df_orders_tF)

for i in range(len(df_orders_tF)):
    coordinates = str(df_orders_tF['pickup_latitude'][i]) + ',' + str(df_orders_tF['pickup_longitude'][i])
    try:
        df_orders_tF['address'][i] = (locator.reverse(coordinates)).raw['address']
    except Exception:
        pass

KeyboardInterrupt: 

time: 1d 7h 22s


<i> And, once again: </i>

In [94]:
df_orders_tF.to_csv('df_orders_tF_218k.csv')

time: 36min 7s


In [49]:
df_orders_tF.head()

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,0 days 00:11:34,11.566667,10-20min,1.6,13.20,9.5,2.20,Standard,CRD,"{'house_number': '4', 'road': 'Vesey Street', ..."
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,0 days 00:26:51,26.850000,20-30min,5.5,27.00,22.0,4.00,Standard,CRD,"{'house_number': '175', 'road': 'Roebling Stre..."
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,...,0 days 00:08:13,8.216667,5-10min,1.6,10.80,8.0,1.80,Standard,CRD,"{'house_number': '25', 'road': 'West 31st Stre..."
3,6433700,5234567812353703,2014-04-12 09:14:58,4,15,2,2014-04-12,Saturday,1,9,...,0 days 00:09:46,9.766667,5-10min,1.3,10.20,8.0,1.70,Standard,CRD,"{'building': 'Fulton Street', 'road': 'Fulton ..."
4,6433701,5234567822243871,2014-04-12 18:59:24,4,15,2,2014-04-12,Saturday,1,18,...,0 days 00:22:07,22.116667,20-30min,3.7,21.25,16.5,4.25,Standard,CRD,"{'amenity': 'Rite Aid', 'house_number': '1849'..."


time: 1.53 s


<i> First results look good!
    <p> Let's check if all the 15,000,000 addresses have been retrieved: </i></p>

In [51]:
df_orders_tF['address'].count()

15000000

time: 1.03 s


In [52]:
i

218209

time: 0 ns


<b><i> Uh oh! Only 218,210 results have been fetched... In 31.5 hours! It means...  <u>90 days</u> for the whole data. Uh oh... </i>

#### 90 x 🕛🕐🕑🕒🕓🕔🕕🕖🕗🕘🕙🕚🕛🕐🕑🕒🕓🕔🕕🕖🕗🕘🕙🕚 <br>

### Reverse geocoding • Nominatim [ numba / njit / numpy vectorize ] ✔️ (kinda... not fast enough)

In [81]:
! pip install numba

time: 51.3 sRequirement already satisfied: numba in c:\users\graca\anaconda3\lib\site-packages (0.51.2)



In [82]:
from numba import njit
import numpy as np

time: 39.7 s


In [83]:
@njit(parallel=True)
def get_address(lat, lng):
    coordinates = str(lat) + ',' + str(lng)
    try:
        address = (locator.reverse(coordinates)).raw['address']
    except Exception:
        pass
    return address

df_orders_tF_sm = df_orders_tF[i:i+100]
df_orders_tF_sm['address'] = get_address(df_orders_tF_sm['pickup_latitude'].values, df_orders_tF_sm['pickup_longitude'].values)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'locator':[0m [1m[1mcannot determine Numba type of <class 'geopy.geocoders.nominatim.Nominatim'>[0m
[1m
File "<ipython-input-83-bf488ac5f1bf>", line 5:[0m
[1mdef get_address(lat, lng):
    <source elided>
    try:
[1m        address = (locator.reverse(coordinates)).raw['address']
[0m        [1m^[0m[0m
[0m

time: 14.6 s


In [84]:
def get_address(lat, lng):
    coordinates = str(lat) + ',' + str(lng)
    try:
        address = (locator.reverse(coordinates)).raw['address']
    except Exception:
        pass
    return address

df_orders_tF_sm = df_orders_tF[i:i+100]
df_orders_tF_sm['address'] = np.vectorize(get_address)(df_orders_tF_sm['pickup_latitude'], df_orders_tF_sm['pickup_longitude'])

time: 52.2 s


In [87]:
df_orders_tF_sm.head(2)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address
218209,6651906,5234567812372677,2014-04-14 01:46:47,4,16,2,2014-04-14,Monday,0,1,...,0 days 00:05:40,5.666667,5-10min,1.1,8.0,6.0,1.0,Standard,CRD,"{'building': 'Symphony House', 'road': 'West 5..."
218210,6651907,5234567816149281,2014-04-14 03:32:11,4,16,2,2014-04-14,Monday,0,3,...,0 days 00:03:05,3.083333,<=5min,0.6,6.6,4.5,1.1,Standard,CRD,"{'house_number': '54', 'road': 'West Houston S..."


time: 31 ms


In [88]:
df_orders_tF_sm.tail(2)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address
218307,6652004,5234567813368025,2014-04-13 23:44:36,4,15,2,2014-04-13,Sunday,1,23,...,0 days 00:14:12,14.200000,10-20min,6.4,21.5,20.5,0.0,Standard,CSH,"{'house_number': '601', 'road': '10th Avenue',..."
218308,6652005,5234567812392667,2014-04-13 18:18:48,4,15,2,2014-04-13,Sunday,1,18,...,0 days 00:15:31,15.516667,10-20min,1.0,11.0,10.5,0.0,Standard,CSH,"{'house_number': '645', 'road': '2nd Avenue', ..."


time: 31 ms


In [90]:
def get_address(lat, lng):
    coordinates = str(lat) + ',' + str(lng)
    try:
        address = (locator.reverse(coordinates)).raw['address']
    except Exception:
        pass
    return address

df_orders_tF['address'] = np.vectorize(get_address)(df_orders_tF['pickup_latitude'], df_orders_tF['pickup_longitude'])

KeyboardInterrupt: 

time: 1h 24min 47s


### Reverse geocoding • Taking Einstein's advice ✔️✔️✔️

#### <i> « As simple as possible... </i>

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Einstein___as_simple_as_possible.jpg'>

#### <i> ... but not simpler » </i>

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Simplicity-Through-Thoughtful-Reduction.png'>

#### Geometrical approach: using points and polynoms to define each borough

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/NYC__Coordinates.png'>

#### Getting boroughs coordinates from Google Maps:

#### <i><u>Staten Island</u></i>:

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/NYC__Staten_Island.png'>

#### <i><u>Bronx</u></i>:

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/NYC__Bronx.png'>

#### <i><u>Queens</u></i>:

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/NYC__Queens1.png'>

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/NYC__Queens2.png'>

#### ( Resuming after Kernel shut down • Retrieving data: nice it'd been saved! 😅 )

In [1]:
%load_ext autotime
import pandas as pd

time: 25.6 s


In [2]:
import warnings
warnings.filterwarnings('ignore')

time: 0 ns


In [3]:
df_orders_tF = pd.read_csv('df_orders_tF_218k.csv', index_col=0) # reading previously transformed data
df_orders_tF.head(2)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,0 days 00:11:34,11.566667,10-20min,1.6,13.2,9.5,2.2,Standard,CRD,"{'house_number': '4', 'road': 'Vesey Street', ..."
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,0 days 00:26:51,26.850000,20-30min,5.5,27.0,22.0,4.0,Standard,CRD,"{'house_number': '175', 'road': 'Roebling Stre..."


time: 6min 26s


#### Defining a « good enough » function to locate the boroughs

In [4]:
import numpy as np

time: 0 ns


In [7]:
def get_borough(lat,lng):
    
    M_Q_B = np.polyfit([40.794346, 40.739384], [-73.914747, -73.9611989], 1) # Manhattan-Queens-Brooklyn line boundary
    Q_B = np.polyfit([40.739384, 40.617010], [-73.9611989, -73.767564], 1) # Queens-Brooklyn line boundary
    
    if lng < -74.052499:
        borough = 'Staten Island'
        
    elif lng > -73.914747 and lat > 40.794346:
        borough = 'Bronx'
        
    elif ((M_Q_B[0] * lat + M_Q_B[1]) - lng < 0): # points (i) 40.794346, -73.914747 (ii) 40.739384, -73.9611989 [ numpy polyfit ]
        if (Q_B[0] * lat + Q_B[1]) - lng < 0: # points (i) 40.739384, -73.9611989 (ii) 40.617010, -73.767564  [ numpy polyfit ]
            borough = 'Queens'
            
        else:
            borough = 'Brooklyn'
            
    else:
        borough = 'Manhattan'
        
    return borough

time: 0 ns


In [8]:
# Testing « get_borough() » for each center point
print('Should be Staten Island: ', get_borough(40.581785, -74.144932), '✅' if get_borough(40.581785, -74.144932) == 'Staten Island' else '❌')
print('Should be Bronx: ', get_borough(40.844984, -73.876748), '✅' if get_borough(40.844984, -73.876748) == 'Bronx' else '❌')
print('Should be Queens: ', get_borough(40.716471, -73.799612), '✅' if get_borough(40.716471, -73.799612) == 'Queens' else '❌')
print('Should be Brooklyn: ', get_borough(40.636348, -73.951523), '✅' if get_borough(40.636348, -73.951523) == 'Brooklyn' else '❌')
print('Should be Manhattan: ', get_borough(40.770309, -73.977383), '✅' if get_borough(40.770309, -73.977383) == 'Manhattan' else '❌')

Should be Staten Island:  Staten Island ✅
Should be Bronx:  Bronx ✅
Should be Queens:  Queens ✅
Should be Brooklyn:  Brooklyn ✅
Should be Manhattan:  Manhattan ✅
time: 375 ms


<i> Great! </i> ✔️✔️✔️

#### Reverse Geocoding: the light way ✔️✔️✔️

In [7]:
df_orders_tF['pickup_borough'] = np.vectorize(get_borough)(df_orders_tF['pickup_latitude'], df_orders_tF['pickup_longitude'])
df_orders_tF['dropoff_borough'] = np.vectorize(get_borough)(df_orders_tF['dropoff_latitude'], df_orders_tF['dropoff_longitude'])
df_orders_tF.head(3)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address,pickup_borough,dropoff_borough
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,10-20min,1.6,13.2,9.5,2.2,Standard,CRD,"{'house_number': '4', 'road': 'Vesey Street', ...",Manhattan,Manhattan
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,20-30min,5.5,27.0,22.0,4.0,Standard,CRD,"{'house_number': '175', 'road': 'Roebling Stre...",Brooklyn,Manhattan
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,...,5-10min,1.6,10.8,8.0,1.8,Standard,CRD,"{'house_number': '25', 'road': 'West 31st Stre...",Manhattan,Manhattan


time: 1h 36min 57s


In [8]:
df_orders_tF.to_csv('df_orders_tF_boroughs.csv')

time: 16min 37s


In [9]:
df_orders_tF.tail(3)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address,pickup_borough,dropoff_borough
14999997,6433694,5234567812370408,2014-04-11 22:43:44,4,15,2,2014-04-11,Friday,0,22,...,NaN,NaN,NaN,NaN,NaN,NaN,CRD,NaN,Manhattan,Manhattan
14999998,6433695,5234567815076100,2014-04-11 19:13:39,4,15,2,2014-04-11,Friday,0,19,...,NaN,NaN,NaN,NaN,NaN,NaN,CRD,NaN,Manhattan,Manhattan
14999999,6433696,5234567812361497,2014-04-11 22:00:42,4,15,2,2014-04-11,Friday,0,22,...,20-30min,6.6,30.62,23.5,6.12,Standard,CRD,NaN,Manhattan,Queens


time: 3.67 s


In [14]:
pd.DataFrame( ( 100 * (df_orders_tF.groupby('pickup_borough')['order_id'].count() / len(df_orders_tF)).apply(lambda x: round(x, 4)) ).sort_values(ascending=False)).rename(columns={'order_id':'%'})

,%
pickup_borough,
Manhattan,91.04
Queens,6.37
Brooklyn,2.52
Bronx,0.04
Staten Island,0.03


time: 9.7 s


In [15]:
pd.DataFrame( ( 100 * (df_orders_tF.groupby('dropoff_borough')['order_id'].count() / len(df_orders_tF)).apply(lambda x: round(x, 4)) ).sort_values(ascending=False)).rename(columns={'order_id':'%'})

,%
dropoff_borough,
Manhattan,90.03
Queens,5.17
Brooklyn,4.32
Bronx,0.31
Staten Island,0.16


time: 1.52 s



💡 <b><i> 3/5 boroughs are related to almost all of the trips: ⮚ <u>90%+</u> both originate / end in <u>Manhattan</u> ⮚ <u>99%+</u> in <u>Manhattan-Queens-Brooklyn</u>. </i></b>

<i> Well... Given its representativeness, it seems useful to further divide Manhattan.
<p> As only 1/5 of the boroughs are being subdivided, instead of creating a new feature, let's just add a suffix to Manhattan. </i>

#### ( Resuming after Kernel shut down • Retrieving data: nice it'd been saved! 😅 )

In [1]:
%load_ext autotime
import pandas as pd
import numpy as np

time: 17.5 s


In [2]:
import warnings
warnings.filterwarnings('ignore')

time: 0 ns


In [3]:
df_orders_tF = pd.read_csv('df_orders_tF_boroughs.csv', index_col=0) # reading previously transformed data
df_orders_tF.head(2)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address,pickup_borough,dropoff_borough
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,10-20min,1.6,13.2,9.5,2.2,Standard,CRD,"{'house_number': '4', 'road': 'Vesey Street', ...",Manhattan,Manhattan
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,20-30min,5.5,27.0,22.0,4.0,Standard,CRD,"{'house_number': '175', 'road': 'Roebling Stre...",Brooklyn,Manhattan


time: 8min 54s


#### Getting coordinates of Manhattan regions from Google Maps:

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Manhattan.png'>

#### <i><u>Washington Heights - Harlem</u></i>:

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Manhattan__HH1.png'>

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Manhattan__HH2.png'>

#### <i><u>Harlem - Upper Manhattan</u></i>:

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Manhattan__HU1.png'>

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Manhattan__HU2.png'>

#### <i><u>Upper Manhattan - Midtown Manhattan</u></i>:

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Manhattan__UM1.png'>

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Manhattan__UM2.png'>

#### <i><u>Midtown Manhattan - Lower Manhattan</u></i>:

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Manhattan__ML1.png'>

<img src='https://raw.githubusercontent.com/IsisSantosCosta/99_Challenge/main/img/Manhattan__ML2.png'>

#### Defining a « good enough » function to locate Manhattan regions

In [16]:
def get_region(lat,lng,borough):
    
    if borough != 'Manhattan':
        return borough
    
    else:
        H_H = np.polyfit([40.834289, 40.828474], [-73.949542, -73.934876], 1) # The Heights-Harlem line boundary
        H_U = np.polyfit([40.819026, 40.783486], [-73.961170, -73.943489], 1) # Harlem-Upper Manhattan line boundary
        U_M = np.polyfit([40.77655 , 40.758549], [-73.993623, -73.959891], 1) # Upper Manhattan-Midtown Manhattan line boundary
        M_L = np.polyfit([40.742163, 40.726814], [-74.008986, -73.972594], 1) # Midtown Manhattan-Lower Manhattan line boundary
        
        if ((H_H[0] * lat + H_H[1]) - lng < 0):
            region = 'Washington Heights'
        
        elif ((H_U[0] * lat + H_U[1]) - lng < 0):
            region = 'Harlem'
            
        elif ((U_M[0] * lat + U_M[1]) - lng < 0):
            region = 'Upper Manhattan'
        
        elif ((M_L[0] * lat + M_L[1]) - lng < 0):
            region = 'Midtown Manhattan'
            
        else:
            region = 'Lower Manhattan'
            
        return borough + ' • ' + region

time: 0 ns


In [18]:
# Testing « get_region() » for geographical coordinates in Manhattan and unchanged for the other boroughs
print('Should be Staten Island: ', get_region(40.581785, -74.144932, 'Staten Island'), '✅' if get_region(40.581785, -74.144932, 'Staten Island') == 'Staten Island' else '❌')
print('Should be Bronx: ', get_region(40.844984, -73.876748, 'Bronx'), '✅' if get_region(40.844984, -73.876748, 'Bronx') == 'Bronx' else '❌')
print('Should be Queens: ', get_region(40.716471, -73.799612, 'Queens'), '✅' if get_region(40.716471, -73.799612, 'Queens') == 'Queens' else '❌')
print('Should be Brooklyn: ', get_region(40.636348, -73.951523, 'Brooklyn'), '✅' if get_region(40.636348, -73.951523, 'Brooklyn') == 'Brooklyn' else '❌')
print('Should be Manhattan • Washington Heights: ', get_region(40.845676, -73.936621, 'Manhattan'), '✅' if get_region(40.845676, -73.936621, 'Manhattan') == 'Manhattan • Washington Heights' else '❌')
print('Should be Manhattan • Harlem: ', get_region(40.814381, -73.942332, 'Manhattan'), '✅' if get_region(40.814381, -73.942332, 'Manhattan') == 'Manhattan • Harlem' else '❌')
print('Should be Manhattan • Upper Manhattan: ', get_region(40.774900, -73.964219, 'Manhattan'), '✅' if get_region(40.774900, -73.964219, 'Manhattan') == 'Manhattan • Upper Manhattan' else '❌')
print('Should be Manhattan • Midtown Manhattan: ', get_region(40.749566, -73.984354, 'Manhattan'), '✅' if get_region(40.749566, -73.984354, 'Manhattan') == 'Manhattan • Midtown Manhattan' else '❌')
print('Should be Manhattan • Lower Manhattan: ', get_region(40.718104, -73.977383, 'Manhattan'), '✅' if get_region(40.718104, -73.977383, 'Manhattan') == 'Manhattan • Lower Manhattan' else '❌')

Should be Staten Island:  Staten Island ✅
Should be Bronx:  Bronx ✅
Should be Queens:  Queens ✅
Should be Brooklyn:  Brooklyn ✅
Should be Manhattan • Washington Heights:  Manhattan • Washington Heights ✅
Should be Manhattan • Harlem:  Manhattan • Harlem ✅
Should be Manhattan • Upper Manhattan:  Manhattan • Upper Manhattan ✅
Should be Manhattan • Midtown Manhattan:  Manhattan • Midtown Manhattan ✅
Should be Manhattan • Lower Manhattan:  Manhattan • Lower Manhattan ✅
time: 16 ms


#### (Further) Reverse Geocoding: Manhattan regions ✔️✔️✔️

In [19]:
df_orders_tF['pickup_borough'] = np.vectorize(get_region)(df_orders_tF['pickup_latitude'], df_orders_tF['pickup_longitude'], df_orders_tF['pickup_borough'])
df_orders_tF['dropoff_borough'] = np.vectorize(get_region)(df_orders_tF['dropoff_latitude'], df_orders_tF['dropoff_longitude'], df_orders_tF['dropoff_borough'])
df_orders_tF.head(3)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address,pickup_borough,dropoff_borough
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,10-20min,1.6,13.2,9.5,2.2,Standard,CRD,"{'house_number': '4', 'road': 'Vesey Street', ...",Manhattan • Lower Manhattan,Manhattan • Lower Manhattan
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,20-30min,5.5,27.0,22.0,4.0,Standard,CRD,"{'house_number': '175', 'road': 'Roebling Stre...",Brooklyn,Manhattan • Midtown Manhattan
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,...,5-10min,1.6,10.8,8.0,1.8,Standard,CRD,"{'house_number': '25', 'road': 'West 31st Stre...",Manhattan • Midtown Manhattan,Manhattan • Midtown Manhattan


time: 3h 13min 13s


In [20]:
df_orders_tF.to_csv('df_orders_tF_boroughs_regions.csv')

time: 19min 40s


In [21]:
df_orders_tF.tail(3)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address,pickup_borough,dropoff_borough
14999997,6433694,5234567812370408,2014-04-11 22:43:44,4,15,2,2014-04-11,Friday,0,22,...,NaN,NaN,NaN,NaN,NaN,NaN,CRD,NaN,Manhattan • Midtown Manhattan,Manhattan • Lower Manhattan
14999998,6433695,5234567815076100,2014-04-11 19:13:39,4,15,2,2014-04-11,Friday,0,19,...,NaN,NaN,NaN,NaN,NaN,NaN,CRD,NaN,Manhattan • Upper Manhattan,Manhattan • Lower Manhattan
14999999,6433696,5234567812361497,2014-04-11 22:00:42,4,15,2,2014-04-11,Friday,0,22,...,20-30min,6.6,30.62,23.5,6.12,Standard,CRD,NaN,Manhattan • Lower Manhattan,Queens


time: 3.44 s


In [22]:
pd.DataFrame( ( 100 * (df_orders_tF.groupby('pickup_borough')['order_id'].count() / len(df_orders_tF)).apply(lambda x: round(x, 4)) ).sort_values(ascending=False)).rename(columns={'order_id':'%'})

,%
pickup_borough,
Manhattan • Midtown Manhattan,46.99
Manhattan • Upper Manhattan,22.99
Manhattan • Lower Manhattan,19.86
Queens,6.37
Brooklyn,2.52
Manhattan • Harlem,1.06
Manhattan • Washington Heights,0.14
Bronx,0.04
Staten Island,0.03


time: 23.7 s


In [23]:
pd.DataFrame( ( 100 * (df_orders_tF.groupby('dropoff_borough')['order_id'].count() / len(df_orders_tF)).apply(lambda x: round(x, 4)) ).sort_values(ascending=False)).rename(columns={'order_id':'%'})

,%
dropoff_borough,
Manhattan • Lower Manhattan,37.18
Manhattan • Midtown Manhattan,33.23
Manhattan • Upper Manhattan,17.40
Queens,5.17
Brooklyn,4.32
Manhattan • Harlem,1.69
Manhattan • Washington Heights,0.53
Bronx,0.31
Staten Island,0.16


time: 26.2 s



💡 <b><i> The Top-3 origins are also the Top-3 destinations, originating slightly more than receiving. </i></b>

## Lessons learned

<ul>
    <li> Volume matters </li>
    <li> Performance matters </li>
    <li> Strategy matters </li>
    <li> This seems not to be the perfect tool </li>
    <li> 15,000,000 records for just 3 months... Better have an approach to deal with multiple months and years </li>
</ul>

## Preparing for the future

#### ( Resuming after Kernel shut down • Retrieving data: nice it'd been saved! 😅 )

In [1]:
%load_ext autotime
import pandas as pd
import numpy as np

time: 19.4 s


In [2]:
import warnings
warnings.filterwarnings('ignore')

time: 0 ns


In [3]:
df_orders_tF = pd.read_csv('df_orders_tF_boroughs_regions.csv', index_col=0) # reading previously transformed data
df_orders_tF.head(2)

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,...,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,address,pickup_borough,dropoff_borough
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,...,10-20min,1.6,13.2,9.5,2.2,Standard,CRD,"{'house_number': '4', 'road': 'Vesey Street', ...",Manhattan • Lower Manhattan,Manhattan • Lower Manhattan
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,...,20-30min,5.5,27.0,22.0,4.0,Standard,CRD,"{'house_number': '175', 'road': 'Roebling Stre...",Brooklyn,Manhattan • Midtown Manhattan


time: 5min 27s


#### Creating aggregated data tables

<i> Considering the last remark above, on cummulative future data, preparing aggregated dataframes is a way to reduce data while keeping much of information. <br>
    Regarding time, <u>daily data</u> is still quite granular in the long run and relatively compact at the same time.<br>
    Aggregation by time shadows information on client, let's thus create also an aggregation by <u>passenger</u>.<br><br>
    First, as </i>'address'<i> had only 'a few' values fetched and it has already been saved as </i>df_orders_tF_218k.csv<i>, let's drop it from the final transformed dataframe.
    <br><i> And let's take a look at all of the columns, without collapsing any: </i>
</i>

In [6]:
pd.set_option('display.max_columns', len(df_orders_tF))
df_orders_tF.drop(columns='address', inplace=True)

time: 1min 32s


In [7]:
df_orders_tF

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,dropoff_datetime,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,pickup_borough,dropoff_borough
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,1,40.714476,-74.015684,40.718173,-74.000814,2014-04-11 18:18:19,0 days 00:11:34,11.566667,10-20min,1.6,13.20,9.5,2.20,Standard,CRD,Manhattan • Lower Manhattan,Manhattan • Lower Manhattan
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,0,40.713247,-73.958403,40.758526,-73.991395,2014-04-12 00:47:19,0 days 00:26:51,26.850000,20-30min,5.5,27.00,22.0,4.00,Standard,CRD,Brooklyn,Manhattan • Midtown Manhattan
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,0,40.747428,-73.987125,40.740963,-74.005073,2014-04-11 23:59:41,0 days 00:08:13,8.216667,5-10min,1.6,10.80,8.0,1.80,Standard,CRD,Manhattan • Midtown Manhattan,Manhattan • Midtown Manhattan
3,6433700,5234567812353703,2014-04-12 09:14:58,4,15,2,2014-04-12,Saturday,1,9,1,40.710255,-74.007610,40.726037,-73.998705,2014-04-12 09:24:44,0 days 00:09:46,9.766667,5-10min,1.3,10.20,8.0,1.70,Standard,CRD,Manhattan • Lower Manhattan,Manhattan • Lower Manhattan
4,6433701,5234567822243871,2014-04-12 18:59:24,4,15,2,2014-04-12,Saturday,1,18,1,40.784171,-73.947220,40.760459,-73.991092,2014-04-12 19:21:31,0 days 00:22:07,22.116667,20-30min,3.7,21.25,16.5,4.25,Standard,CRD,Manhattan • Upper Manhattan,Manhattan • Midtown Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14999995,6433692,5234567812366923,2014-04-11 22:47:18,4,15,2,2014-04-11,Friday,0,22,0,40.742317,-73.996942,40.727788,-74.003086,2014-04-11 22:53:16,0 days 00:05:58,5.966667,5-10min,1.2,8.75,6.0,1.75,Standard,CRD,Manhattan • Midtown Manhattan,Manhattan • Lower Manhattan
14999996,6433693,5234567818373508,2014-04-11 20:27:48,4,15,2,2014-04-11,Friday,0,20,1,40.734323,-74.003060,40.737984,-73.987018,2014-04-11 20:44:28,0 days 00:16:40,16.666667,10-20min,1.3,14.40,11.0,2.40,Standard,CRD,Manhattan • Lower Manhattan,Manhattan • Midtown Manhattan
14999997,6433694,5234567812370408,2014-04-11 22:43:44,4,15,2,2014-04-11,Friday,0,22,0,40.748209,-73.988538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRD,Manhattan • Midtown Manhattan,Manhattan • Lower Manhattan
14999998,6433695,5234567815076100,2014-04-11 19:13:39,4,15,2,2014-04-11,Friday,0,19,1,40.785448,-73.976680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRD,Manhattan • Upper Manhattan,Manhattan • Lower Manhattan


time: 1.53 s


#### Daily data

In [10]:
df_orders_tF_daily = df_orders_tF.groupby('pickup_date').mean()
df_orders_tF_daily

,order_id,passenger_id,pickup_month,pickup_week,pickup_weekinmonth,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,trip_distance,total_amount,fare_amount,tip_amount
pickup_date,,,,,,,,,,,,,,,,,
2014-03-01,4.897605e+06,5.234568e+15,3.0,9.0,1.0,1.0,13.127753,0.425838,40.084363,-72.772283,40.060088,-72.725061,12.191076,2.766199,14.058896,11.856336,1.333832
2014-03-02,4.930377e+06,5.234568e+15,3.0,9.0,1.0,1.0,11.634141,0.409126,40.099519,-72.797495,40.053491,-72.709109,11.277417,3.103580,14.641738,12.269199,1.421354
2014-03-03,5.047677e+06,5.234568e+15,3.0,10.0,1.0,0.0,13.862015,0.565459,40.070077,-72.737101,40.023182,-72.651435,10.800999,2.723376,13.783595,11.286525,1.392459
2014-03-04,5.091889e+06,5.234568e+15,3.0,10.0,1.0,0.0,14.178427,0.555079,40.059002,-72.717296,40.014721,-72.635930,12.413448,2.700972,14.401247,11.802064,1.489682
2014-03-05,5.202453e+06,5.234568e+15,3.0,10.0,1.0,0.0,14.114823,0.542031,40.107828,-72.806730,40.066120,-72.729210,12.738018,2.770363,14.769655,12.107511,1.535411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-05-25,4.517509e+06,5.234568e+15,5.0,21.0,4.0,1.0,11.424132,0.351337,39.979245,-72.584938,40.027577,-72.667116,12.279550,3.358891,16.479633,13.220956,2.250033
2014-05-28,3.886191e+06,5.234568e+15,5.0,22.0,4.0,0.0,12.500000,0.166667,40.754782,-73.988229,40.754421,-73.984579,12.285714,0.981429,10.671429,9.071429,1.100000
2014-05-29,3.886160e+06,5.234568e+15,5.0,22.0,5.0,0.0,17.500000,0.300000,40.748109,-73.982711,40.742318,-73.982522,12.176471,2.061176,12.952941,10.411765,1.374706


time: 1min 1s


In [19]:
# order_id can become total_orders, and instead of the mean, show the result for .count()
df_orders_tF_daily.rename(columns={'order_id':'total_orders'}, inplace=True)
df_orders_tF_daily['total_orders'] = df_orders_tF.groupby('pickup_date').count()
print('Sum of daily orders: ', sum(df_orders_tF_daily['total_orders']))

# similarly, passenger_id gives place to total_trips, by counting any column for coverted orders. Let's take trip_distance
df_orders_tF_daily.rename(columns={'passenger_id':'total_trips'}, inplace=True)
df_orders_tF_daily['total_trips'] = df_orders_tF.groupby('pickup_date')['trip_distance'].count()
print('Sum of daily trips: ', sum(df_orders_tF_daily['total_trips']))

# columns pickup_month to pickup_isweekend should be integer
df_orders_tF_daily[['pickup_month','pickup_week','pickup_weekinmonth','pickup_isweekend']] = df_orders_tF_daily[['pickup_month','pickup_week','pickup_weekinmonth','pickup_isweekend']].astype('int')

# let's see how it looks like
df_orders_tF_daily.head(3)

Sum of daily orders:  15000000
Sum of daily trips:  11456227


,total_orders,total_trips,pickup_month,pickup_week,pickup_weekinmonth,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,trip_distance,total_amount,fare_amount,tip_amount
pickup_date,,,,,,,,,,,,,,,,,
2014-03-01,277284,211545,3,9,1,1,13.127753,0.425838,40.084363,-72.772283,40.060088,-72.725061,12.191076,2.766199,14.058896,11.856336,1.333832
2014-03-02,231264,176557,3,9,1,1,11.634141,0.409126,40.099519,-72.797495,40.053491,-72.709109,11.277417,3.103580,14.641738,12.269199,1.421354
2014-03-03,232098,177632,3,10,1,0,13.862015,0.565459,40.070077,-72.737101,40.023182,-72.651435,10.800999,2.723376,13.783595,11.286525,1.392459


time: 1min 12s


<br><i> Categorical data missing are </i>'pickup_dayofweek', 'duration_type', 'rate_type', 'payment_type', 'pickup_borough', 'dropoff_borough'.<br>
<i>For the chosen aggregation, 'pickup_dayofweek' is unique, all the others may be broken into their classes, with the values summing up their frequency:</i>

In [35]:
# pickup_dayofweek
dayofweek_dict= {'0': 'Monday'}
dayofweek_dict['1'] = 'Tuesday'
dayofweek_dict['2'] = 'Wednesday'
dayofweek_dict['3'] = 'Thursday'
dayofweek_dict['4'] = 'Friday'
dayofweek_dict['5'] = 'Saturday'
dayofweek_dict['6'] = 'Sunday'

dayofweek_value = pd.to_datetime(pd.DatetimeIndex(df_orders_tF_daily.index).dayofweek).astype(str)
df_orders_tF_daily.insert(loc=5, column='pickup_dayofweek', value = dayofweek_value)
df_orders_tF_daily['pickup_dayofweek'] = df_orders_tF_daily['pickup_dayofweek'].str[-1:]
df_orders_tF_daily['pickup_dayofweek'] = df_orders_tF_daily['pickup_dayofweek'].replace(dayofweek_dict)

df_orders_tF_daily.head(1)

,total_orders,total_trips,pickup_month,pickup_week,pickup_weekinmonth,pickup_dayofweek,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,trip_distance,total_amount,fare_amount,tip_amount
pickup_date,,,,,,,,,,,,,,,,,,
2014-03-01,277284,211545,3,9,1,Saturday,1,13.127753,0.425838,40.084363,-72.772283,40.060088,-72.725061,12.191076,2.766199,14.058896,11.856336,1.333832


time: 109 ms


In [44]:
# duration_type
durationtype_dict= {max(df_orders_tF_daily['duration_minutes']): '>60min'}
durationtype_dict[60] = '40-60min'
durationtype_dict[40] = '30-40min'
durationtype_dict[30] = '20-30min'
durationtype_dict[20] = '10-20min'
durationtype_dict[10] = '5-10min'
durationtype_dict[ 5] = '<=5min'

df_orders_tF_daily.insert(loc=14, column='duration_type', value = df_orders_tF_daily['duration_minutes'])

for i in range(len(durationtype_dict)):
    df_orders_tF_daily.loc[df_orders_tF_daily['duration_minutes'] <= int(list(durationtype_dict)[i]), 'duration_type'] = list(durationtype_dict.values())[i]

df_orders_tF_daily.head(1)

,total_orders,total_trips,pickup_month,pickup_week,pickup_weekinmonth,pickup_dayofweek,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount
pickup_date,,,,,,,,,,,,,,,,,,,
2014-03-01,277284,211545,3,9,1,Saturday,1,13.127753,0.425838,40.084363,-72.772283,40.060088,-72.725061,12.191076,10-20min,2.766199,14.058896,11.856336,1.333832


time: 9.92 s


In [93]:
# rate_type

rate_types = [
    'Standard',
    'JFK',
    'Newark',
    'Nassau / Westchester',
    'Negotiated',
    'Group ride']

total = 0
    
for rate_type in rate_types:
    df_orders_tF_daily[rate_type] = df_orders_tF.loc[df_orders_tF['rate_type'] == rate_type, ['pickup_date','rate_type']].groupby('pickup_date').count()
    df_orders_tF_daily[rate_type].fillna(0, inplace=True)
    df_orders_tF_daily[rate_type] = df_orders_tF_daily[rate_type].astype('int')
    print(rate_type + ': ', df_orders_tF_daily[rate_type].sum())
    total += df_orders_tF_daily[rate_type].sum()
    
print('Total: ', total)

df_orders_tF_daily.head(3)

Standard:  11196413
JFK:  207614
Newark:  17871
Nassau / Westchester:  3617
Negotiated:  30557
Group ride:  155
Total:  11456227


,total_orders,total_trips,pickup_month,pickup_week,pickup_weekinmonth,pickup_dayofweek,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,Standard,JFK,Newark,Nassau / Westchester,Negotiated,Group ride
pickup_date,,,,,,,,,,,,,,,,,,,,,,,,,
2014-03-01,277284,211545,3,9,1,Saturday,1,13.127753,0.425838,40.084363,-72.772283,40.060088,-72.725061,12.191076,10-20min,2.766199,14.058896,11.856336,1.333832,208065,2672,229,39,540,0
2014-03-02,231264,176557,3,9,1,Sunday,1,11.634141,0.409126,40.099519,-72.797495,40.053491,-72.709109,11.277417,10-20min,3.103580,14.641738,12.269199,1.421354,172260,3455,295,54,486,7
2014-03-03,232098,177632,3,10,1,Monday,0,13.862015,0.565459,40.070077,-72.737101,40.023182,-72.651435,10.800999,10-20min,2.723376,13.783595,11.286525,1.392459,174170,2760,227,50,422,3


time: 1min 20s


In [95]:
# payment_type

payment_types = [
    'CRD', # credit card
    'CSH', # cash
    'UNK', # unknown
    'NOC', # no charge
    'DIS'] # dispute

total = 0
    
for payment_type in payment_types:
    df_orders_tF_daily[payment_type] = df_orders_tF.loc[df_orders_tF['payment_type'] == payment_type, ['pickup_date','payment_type']].groupby('pickup_date').count()
    df_orders_tF_daily[payment_type].fillna(0, inplace=True)
    df_orders_tF_daily[payment_type] = df_orders_tF_daily[payment_type].astype('int')
    print(payment_type + ': ', df_orders_tF_daily[payment_type].sum())
    total += df_orders_tF_daily[payment_type].sum()
    
print('Total: ', total)

df_orders_tF_daily.head(3)

CRD:  8642716
CSH:  6232474
UNK:  74282
NOC:  38786
DIS:  11742
Total:  15000000


,total_orders,total_trips,pickup_month,pickup_week,pickup_weekinmonth,pickup_dayofweek,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,Standard,JFK,Newark,Nassau / Westchester,Negotiated,Group ride,CRD,CSH,UNK,NOC,DIS
pickup_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-03-01,277284,211545,3,9,1,Saturday,1,13.127753,0.425838,40.084363,-72.772283,40.060088,-72.725061,12.191076,10-20min,2.766199,14.058896,11.856336,1.333832,208065,2672,229,39,540,0,155788,119182,1414,714,186
2014-03-02,231264,176557,3,9,1,Sunday,1,11.634141,0.409126,40.099519,-72.797495,40.053491,-72.709109,11.277417,10-20min,3.103580,14.641738,12.269199,1.421354,172260,3455,295,54,486,7,132324,97072,1100,620,148
2014-03-03,232098,177632,3,10,1,Monday,0,13.862015,0.565459,40.070077,-72.737101,40.023182,-72.651435,10.800999,10-20min,2.723376,13.783595,11.286525,1.392459,174170,2760,227,50,422,3,134902,95372,1212,508,104


time: 1min 7s


In [97]:
# pickup_borough

pickup_boroughs = [
    'Manhattan • Lower Manhattan',
    'Manhattan • Midtown Manhattan',
    'Manhattan • Upper Manhattan',
    'Queens',
    'Brooklyn',
    'Manhattan • Harlem',
    'Manhattan • Washington Heights',
    'Bronx',
    'Staten Island']

total = 0
    
for pickup_borough in pickup_boroughs:
    df_orders_tF_daily['pickup ‖ ' + pickup_borough] = df_orders_tF.loc[df_orders_tF['pickup_borough'] == pickup_borough, ['pickup_date','pickup_borough']].groupby('pickup_date').count()
    df_orders_tF_daily['pickup ‖ ' + pickup_borough].fillna(0, inplace=True)
    df_orders_tF_daily['pickup ‖ ' + pickup_borough] = df_orders_tF_daily['pickup ‖ ' + pickup_borough].astype('int')
    print(pickup_borough + ': ', df_orders_tF_daily['pickup ‖ ' + pickup_borough].sum())
    total += df_orders_tF_daily['pickup ‖ ' + pickup_borough].sum()
    
print('Total: ', total)

df_orders_tF_daily.head(3)

Manhattan • Lower Manhattan:  2979020
Manhattan • Midtown Manhattan:  7048276
Manhattan • Upper Manhattan:  3448762
Queens:  955922
Brooklyn:  377326
Manhattan • Harlem:  159468
Manhattan • Washington Heights:  20968
Bronx:  6066
Staten Island:  4192
Total:  15000000


,total_orders,total_trips,pickup_month,pickup_week,pickup_weekinmonth,pickup_dayofweek,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,Standard,JFK,Newark,Nassau / Westchester,Negotiated,Group ride,CRD,CSH,UNK,NOC,DIS,pickup ‖ Manhattan • Lower Manhattan,pickup ‖ Manhattan • Midtown Manhattan,pickup ‖ Manhattan • Upper Manhattan,pickup ‖ Queens,pickup ‖ Brooklyn,pickup ‖ Manhattan • Harlem,pickup ‖ Manhattan • Washington Heights,pickup ‖ Bronx,pickup ‖ Staten Island
pickup_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-03-01,277284,211545,3,9,1,Saturday,1,13.127753,0.425838,40.084363,-72.772283,40.060088,-72.725061,12.191076,10-20min,2.766199,14.058896,11.856336,1.333832,208065,2672,229,39,540,0,155788,119182,1414,714,186,65792,121442,59706,13056,12884,3762,442,148,52
2014-03-02,231264,176557,3,9,1,Sunday,1,11.634141,0.409126,40.099519,-72.797495,40.053491,-72.709109,11.277417,10-20min,3.103580,14.641738,12.269199,1.421354,172260,3455,295,54,486,7,132324,97072,1100,620,148,53024,100852,48142,14648,10634,3260,508,136,60
2014-03-03,232098,177632,3,10,1,Monday,0,13.862015,0.565459,40.070077,-72.737101,40.023182,-72.651435,10.800999,10-20min,2.723376,13.783595,11.286525,1.392459,174170,2760,227,50,422,3,134902,95372,1212,508,104,39726,114252,56120,14466,4808,2342,266,86,32


time: 1min 16s


In [101]:
# dropoff_borough

dropoff_boroughs = [
    'Manhattan • Lower Manhattan',
    'Manhattan • Midtown Manhattan',
    'Manhattan • Upper Manhattan',
    'Queens',
    'Brooklyn',
    'Manhattan • Harlem',
    'Manhattan • Washington Heights',
    'Bronx',
    'Staten Island']

total = 0
    
for dropoff_borough in dropoff_boroughs:
    df_orders_tF_daily['dropoff ‖ ' + dropoff_borough] = df_orders_tF.loc[df_orders_tF['dropoff_borough'] == dropoff_borough, ['pickup_date','dropoff_borough']].groupby('pickup_date').count()
    df_orders_tF_daily['dropoff ‖ ' + dropoff_borough].fillna(0, inplace=True)
    df_orders_tF_daily['dropoff ‖ ' + dropoff_borough] = df_orders_tF_daily['dropoff ‖ ' + dropoff_borough].astype('int')
    print(dropoff_borough + ': ', df_orders_tF_daily['dropoff ‖ ' + dropoff_borough].sum())
    total += df_orders_tF_daily['dropoff ‖ ' + dropoff_borough].sum()
    
print('Total: ', total)

df_orders_tF_daily.head(3)

Manhattan • Lower Manhattan:  5577549
Manhattan • Midtown Manhattan:  4983773
Manhattan • Upper Manhattan:  2609985
Queens:  776089
Brooklyn:  648372
Manhattan • Harlem:  253940
Manhattan • Washington Heights:  79236
Bronx:  46935
Staten Island:  24121
Total:  15000000


,total_orders,total_trips,pickup_month,pickup_week,pickup_weekinmonth,pickup_dayofweek,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,Standard,JFK,Newark,Nassau / Westchester,Negotiated,Group ride,CRD,CSH,UNK,NOC,DIS,pickup ‖ Manhattan • Lower Manhattan,pickup ‖ Manhattan • Midtown Manhattan,pickup ‖ Manhattan • Upper Manhattan,pickup ‖ Queens,pickup ‖ Brooklyn,pickup ‖ Manhattan • Harlem,pickup ‖ Manhattan • Washington Heights,pickup ‖ Bronx,pickup ‖ Staten Island,dropoff ‖ Manhattan • Lower Manhattan,dropoff ‖ Manhattan • Midtown Manhattan,dropoff ‖ Manhattan • Upper Manhattan,dropoff ‖ Queens,dropoff ‖ Brooklyn,dropoff ‖ Manhattan • Harlem,dropoff ‖ Manhattan • Washington Heights,dropoff ‖ Bronx,dropoff ‖ Staten Island
pickup_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-03-01,277284,211545,3,9,1,Saturday,1,13.127753,0.425838,40.084363,-72.772283,40.060088,-72.725061,12.191076,10-20min,2.766199,14.058896,11.856336,1.333832,208065,2672,229,39,540,0,155788,119182,1414,714,186,65792,121442,59706,13056,12884,3762,442,148,52,111019,84961,44495,12251,16583,5226,1426,1030,293
2014-03-02,231264,176557,3,9,1,Sunday,1,11.634141,0.409126,40.099519,-72.797495,40.053491,-72.709109,11.277417,10-20min,3.103580,14.641738,12.269199,1.421354,172260,3455,295,54,486,7,132324,97072,1100,620,148,53024,100852,48142,14648,10634,3260,508,136,60,87641,69577,38391,13171,14763,4806,1602,951,362
2014-03-03,232098,177632,3,10,1,Monday,0,13.862015,0.565459,40.070077,-72.737101,40.023182,-72.651435,10.800999,10-20min,2.723376,13.783595,11.286525,1.392459,174170,2760,227,50,422,3,134902,95372,1212,508,104,39726,114252,56120,14466,4808,2342,266,86,32,83235,80741,42974,10432,9076,3554,1219,583,284


time: 2min 48s


#### Revising dropoff boroughs

<i> As the get_borough and get_region didn't checked for NaN values of latitude and longitude, it has defaulted to else values (Manhattan / Lower Manhattan) for non converted orders. <br><br>
    Instead of calling them again, let's just replace at this point and rerun the dropoff_borough aggregation:

In [103]:
df_orders_tF.loc[np.isnan(df_orders_tF['dropoff_latitude']), 'dropoff_borough'] = float('NaN')
df_orders_tF

,order_id,passenger_id,pickup_datetime,pickup_month,pickup_week,pickup_weekinmonth,pickup_date,pickup_dayofweek,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,dropoff_datetime,duration,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,rate_type,payment_type,pickup_borough,dropoff_borough
0,6433697,5234567816269547,2014-04-11 18:06:45,4,15,2,2014-04-11,Friday,0,18,1,40.714476,-74.015684,40.718173,-74.000814,2014-04-11 18:18:19,0 days 00:11:34,11.566667,10-20min,1.6,13.20,9.5,2.20,Standard,CRD,Manhattan • Lower Manhattan,Manhattan • Lower Manhattan
1,6433698,5234567812367422,2014-04-12 00:20:28,4,15,2,2014-04-12,Saturday,1,0,0,40.713247,-73.958403,40.758526,-73.991395,2014-04-12 00:47:19,0 days 00:26:51,26.850000,20-30min,5.5,27.00,22.0,4.00,Standard,CRD,Brooklyn,Manhattan • Midtown Manhattan
2,6433699,5234567814730471,2014-04-11 23:51:28,4,15,2,2014-04-11,Friday,0,23,0,40.747428,-73.987125,40.740963,-74.005073,2014-04-11 23:59:41,0 days 00:08:13,8.216667,5-10min,1.6,10.80,8.0,1.80,Standard,CRD,Manhattan • Midtown Manhattan,Manhattan • Midtown Manhattan
3,6433700,5234567812353703,2014-04-12 09:14:58,4,15,2,2014-04-12,Saturday,1,9,1,40.710255,-74.007610,40.726037,-73.998705,2014-04-12 09:24:44,0 days 00:09:46,9.766667,5-10min,1.3,10.20,8.0,1.70,Standard,CRD,Manhattan • Lower Manhattan,Manhattan • Lower Manhattan
4,6433701,5234567822243871,2014-04-12 18:59:24,4,15,2,2014-04-12,Saturday,1,18,1,40.784171,-73.947220,40.760459,-73.991092,2014-04-12 19:21:31,0 days 00:22:07,22.116667,20-30min,3.7,21.25,16.5,4.25,Standard,CRD,Manhattan • Upper Manhattan,Manhattan • Midtown Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14999995,6433692,5234567812366923,2014-04-11 22:47:18,4,15,2,2014-04-11,Friday,0,22,0,40.742317,-73.996942,40.727788,-74.003086,2014-04-11 22:53:16,0 days 00:05:58,5.966667,5-10min,1.2,8.75,6.0,1.75,Standard,CRD,Manhattan • Midtown Manhattan,Manhattan • Lower Manhattan
14999996,6433693,5234567818373508,2014-04-11 20:27:48,4,15,2,2014-04-11,Friday,0,20,1,40.734323,-74.003060,40.737984,-73.987018,2014-04-11 20:44:28,0 days 00:16:40,16.666667,10-20min,1.3,14.40,11.0,2.40,Standard,CRD,Manhattan • Lower Manhattan,Manhattan • Midtown Manhattan
14999997,6433694,5234567812370408,2014-04-11 22:43:44,4,15,2,2014-04-11,Friday,0,22,0,40.748209,-73.988538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRD,Manhattan • Midtown Manhattan,NaN
14999998,6433695,5234567815076100,2014-04-11 19:13:39,4,15,2,2014-04-11,Friday,0,19,1,40.785448,-73.976680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRD,Manhattan • Upper Manhattan,NaN


time: 3.88 s


In [106]:
df_orders_tF.to_csv('df_orders_tF.csv')

time: 6min 10s


In [104]:
# dropoff_borough

dropoff_boroughs = [
    'Manhattan • Lower Manhattan',
    'Manhattan • Midtown Manhattan',
    'Manhattan • Upper Manhattan',
    'Queens',
    'Brooklyn',
    'Manhattan • Harlem',
    'Manhattan • Washington Heights',
    'Bronx',
    'Staten Island']

total = 0
    
for dropoff_borough in dropoff_boroughs:
    df_orders_tF_daily['dropoff ‖ ' + dropoff_borough] = df_orders_tF.loc[df_orders_tF['dropoff_borough'] == dropoff_borough, ['pickup_date','dropoff_borough']].groupby('pickup_date').count()
    df_orders_tF_daily['dropoff ‖ ' + dropoff_borough].fillna(0, inplace=True)
    df_orders_tF_daily['dropoff ‖ ' + dropoff_borough] = df_orders_tF_daily['dropoff ‖ ' + dropoff_borough].astype('int')
    print(dropoff_borough + ': ', df_orders_tF_daily['dropoff ‖ ' + dropoff_borough].sum())
    total += df_orders_tF_daily['dropoff ‖ ' + dropoff_borough].sum()
    
print('Total: ', total)

df_orders_tF_daily.head(3)

Manhattan • Lower Manhattan:  2033776
Manhattan • Midtown Manhattan:  4983773
Manhattan • Upper Manhattan:  2609985
Queens:  776089
Brooklyn:  648372
Manhattan • Harlem:  253940
Manhattan • Washington Heights:  79236
Bronx:  46935
Staten Island:  24121
Total:  11456227


,total_orders,total_trips,pickup_month,pickup_week,pickup_weekinmonth,pickup_dayofweek,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,duration_type,trip_distance,total_amount,fare_amount,tip_amount,Standard,JFK,Newark,Nassau / Westchester,Negotiated,Group ride,CRD,CSH,UNK,NOC,DIS,pickup ‖ Manhattan • Lower Manhattan,pickup ‖ Manhattan • Midtown Manhattan,pickup ‖ Manhattan • Upper Manhattan,pickup ‖ Queens,pickup ‖ Brooklyn,pickup ‖ Manhattan • Harlem,pickup ‖ Manhattan • Washington Heights,pickup ‖ Bronx,pickup ‖ Staten Island,dropoff ‖ Manhattan • Lower Manhattan,dropoff ‖ Manhattan • Midtown Manhattan,dropoff ‖ Manhattan • Upper Manhattan,dropoff ‖ Queens,dropoff ‖ Brooklyn,dropoff ‖ Manhattan • Harlem,dropoff ‖ Manhattan • Washington Heights,dropoff ‖ Bronx,dropoff ‖ Staten Island
pickup_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-03-01,277284,211545,3,9,1,Saturday,1,13.127753,0.425838,40.084363,-72.772283,40.060088,-72.725061,12.191076,10-20min,2.766199,14.058896,11.856336,1.333832,208065,2672,229,39,540,0,155788,119182,1414,714,186,65792,121442,59706,13056,12884,3762,442,148,52,45280,84961,44495,12251,16583,5226,1426,1030,293
2014-03-02,231264,176557,3,9,1,Sunday,1,11.634141,0.409126,40.099519,-72.797495,40.053491,-72.709109,11.277417,10-20min,3.103580,14.641738,12.269199,1.421354,172260,3455,295,54,486,7,132324,97072,1100,620,148,53024,100852,48142,14648,10634,3260,508,136,60,32934,69577,38391,13171,14763,4806,1602,951,362
2014-03-03,232098,177632,3,10,1,Monday,0,13.862015,0.565459,40.070077,-72.737101,40.023182,-72.651435,10.800999,10-20min,2.723376,13.783595,11.286525,1.392459,174170,2760,227,50,422,3,134902,95372,1212,508,104,39726,114252,56120,14466,4808,2342,266,86,32,28769,80741,42974,10432,9076,3554,1219,583,284


time: 55.9 s


<i> Fine!</i> ✔️✔️✔️

In [105]:
df_orders_tF_daily.to_csv('df_orders_tF_daily.csv')

time: 2.08 s


#### Data by passenger

In [107]:
df_orders_tF_passenger = df_orders_tF.groupby('passenger_id').mean()
df_orders_tF_passenger

,order_id,pickup_month,pickup_week,pickup_weekinmonth,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,trip_distance,total_amount,fare_amount,tip_amount
passenger_id,,,,,,,,,,,,,,,,
5234567812345678,8.466249e+06,3.693750,14.468750,2.718750,0.250000,13.493750,0.481250,40.242022,-73.052296,40.404677,-73.347880,12.556356,2.509576,13.457712,11.368644,1.065169
5234567812345679,7.201220e+06,3.689394,14.484848,2.727273,0.295455,14.469697,0.469697,40.443271,-73.415126,40.355975,-73.257267,13.465372,3.031456,16.101359,13.218447,1.701845
5234567812345680,7.381015e+06,3.664234,14.138686,2.532847,0.248175,13.189781,0.496350,38.371987,-69.651205,38.790243,-70.418258,12.883814,2.993462,14.478365,12.149038,1.249327
5234567812345681,7.746686e+06,3.721893,14.396450,2.568047,0.325444,14.171598,0.396450,39.304576,-71.351899,38.872119,-70.553578,12.934359,3.129077,16.858308,14.023077,1.616154
5234567812345682,7.468757e+06,3.784173,14.805755,2.611511,0.215827,13.021583,0.482014,40.752774,-73.974381,40.390405,-73.306947,13.261607,3.306607,15.846429,13.169643,1.334821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5234567824345667,6.336512e+06,3.666667,14.333333,2.666667,0.333333,11.000000,0.333333,40.763095,-73.968483,40.781681,-73.965047,6.700000,2.143333,9.500000,9.000000,0.000000
5234567824345668,8.432835e+06,3.000000,13.000000,5.000000,1.000000,18.000000,1.000000,40.749328,-73.992150,40.763977,-73.988155,8.633333,1.300000,8.000000,7.500000,0.000000
5234567824345669,6.698884e+06,3.666667,14.000000,2.333333,0.333333,12.000000,0.000000,40.791566,-73.956707,40.773999,-73.961890,18.666667,2.426667,15.033333,13.666667,0.866667


time: 3min 9s


In [109]:
# order_id can become total_orders, and instead of the mean, show the result for .count()
df_orders_tF_passenger.rename(columns={'order_id':'total_orders'}, inplace=True)
df_orders_tF_passenger['total_orders'] = df_orders_tF.groupby('passenger_id').count()
print('Sum of orders: ', sum(df_orders_tF_passenger['total_orders']))

# let's see how it looks like
df_orders_tF_passenger.head(3)

Sum of orders:  15000000


,total_orders,pickup_month,pickup_week,pickup_weekinmonth,pickup_isweekend,pickup_hour,pickup_isrush,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,duration_minutes,trip_distance,total_amount,fare_amount,tip_amount
passenger_id,,,,,,,,,,,,,,,,
5234567812345678,160,3.693750,14.468750,2.718750,0.250000,13.493750,0.481250,40.242022,-73.052296,40.404677,-73.347880,12.556356,2.509576,13.457712,11.368644,1.065169
5234567812345679,132,3.689394,14.484848,2.727273,0.295455,14.469697,0.469697,40.443271,-73.415126,40.355975,-73.257267,13.465372,3.031456,16.101359,13.218447,1.701845
5234567812345680,137,3.664234,14.138686,2.532847,0.248175,13.189781,0.496350,38.371987,-69.651205,38.790243,-70.418258,12.883814,2.993462,14.478365,12.149038,1.249327


time: 9min 59s


<i> These aggregated data are a first trial, to be refined after EDA and deep dive analysis, as needed. </i>

## Exporting results for retrieval

In [110]:
df_orders_tF_daily.to_csv('df_orders_tF_daily.csv')

time: 5.75 s


In [111]:
df_orders_tF_passenger.to_csv('df_orders_tF_passenger.csv')

time: 5min 5s


In [112]:
df_orders_tF.to_csv('df_orders_tF.csv')

time: 21min 31s


<i>Note: This Notebook (Transformation) would entail the whole transforming process, but as some instabilities may interrupt the Kernel requiring a restart from zero, it has been split for a smoothier process.</i>

💡 <b><i> Aggregating the data can speed up the saving process from 21min to 5min or even to 5s. </i> 😎

# Next: [EDA](99__EDA.ipynb)